In [106]:
import PIL
import pandas as pd
import numpy as np
import tensorflow as tf
import datasets
import os
from dotenv import load_dotenv
from sklearn.utils import class_weight
from sklearn.datasets import load_files
import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import wandb

In [ ]:
#this code only can run after import the wandb
from wandb.integration.keras import WandbModelCheckpoint, WandbMetricsLogger

In [11]:
ds = datasets.load_dataset("garythung/trashnet", split=datasets.ReadInstruction("train", from_=2527, to=5054, unit="abs"))

In [ ]:
#load path on .env
load_dotenv()

True

## Data Preparation

In [ ]:
#create metadata
job_type = "train_model"
config = {
    "optimizer": "adam",
    "loss": "categorical_crossentropy",
    "metrics": ["acc"],
    "epochs": 100,
    "validation_split": 0.1,
}

#initialization wandb for create the run session
run = wandb.init(
    project="trashnet-classification",
    job_type=job_type,
    config=config,
)

#access the latest dataset version with artifact then dowload
version = "latest"
name = "{}:{}".format("{}_dataset".format("trashnet"), version)
artifact = run.use_artifact(artifact_or_name=name)

artifact_dir = artifact.download()




wandb:   2528 of 2528 files downloaded.  


In [144]:
artifact_data=os.path.join(os.getcwd(),os.getenv("ARTIFACT_DIR"))

In [104]:
#get the config of metadata model versioning 
input_shape = (224, 224, 3)
loss = "categorical_crossentropy"
optimizer = run.config["optimizer"]
metrics = ["accuracy"]
epochs = run.config["epochs"]
validation_split = run.config["validation_split"]

In [102]:
#Generate tensorflow/keras dataset format from downloades artifacts dataset version 

#Generate train dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
  artifact_data,
  validation_split=validation_split, #split by 90:10
  subset="training",
  seed=123, #make shuffle dataset from next generate
  image_size=(224, 224),
  batch_size=32,
  label_mode="categorical") #make sure label_mode on categorical so the target shape will (None, num_classes)

#Generate validation/test dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
  artifact_data,
  validation_split=validation_split,#split by 90:10
  subset="validation",
  seed=123, #make shuffle dataset from next generate
  image_size=(224, 224),
  batch_size=32,
  label_mode="categorical")#make sure label_mode on categorical so the target shape will (None, num_classes)


Found 2527 files belonging to 6 classes.
Using 2275 files for training.
Found 2527 files belonging to 6 classes.
Using 252 files for validation.


In [103]:
n_classes = len(train_ds.class_names) #count the number of labels

In [105]:
def CNN_Model(input_shape,classes):
    #Generate manual augmentation layer withou generator
    data_augmentation = tf.keras.Sequential([
        layers.RandomRotation(0.4),
        layers.RandomTranslation(0.2, 0.2),
        layers.RandomZoom(0.2),
        layers.RandomFlip("horizontal"),
        layers.Rescaling(1./255)
    ])
    X_Input = layers.Input(input_shape)
    X = data_augmentation(X_Input) #augmented data before convulation

    #Convolution blocks
    X = layers.Conv2D(32,(3,3), padding='same',activation='relu')(X)
    X = layers.MaxPooling2D(pool_size=2)(X)

    X = layers.Conv2D(64,(3,3), padding='same',activation='relu')(X)
    X =layers.MaxPooling2D(pool_size=2)(X)

    X = layers.Conv2D(32,(3,3), padding='same',activation='relu')(X)
    X = layers.MaxPooling2D(pool_size=2)(X)

    #Classification layers
    X = layers.Flatten()(X)

    X = layers.Dense(64,activation='relu')(X)

    X = layers.Dropout(0.2)(X)
    X = layers.Dense(32,activation='relu')(X)

    X = layers.Dropout(0.2)(X)
    X = layers.Dense(classes,activation='softmax')(X)

    model = Model(inputs=X_Input,outputs=X, name="CNN")
    return model


In [107]:
#build the architecure model
model = CNN_Model(input_shape, n_classes) 

In [123]:
#compile the model with the optimizer, loss, and metric config
model.compile(optimizer=optimizer, loss=loss, metrics=metrics) 

In [ ]:
#show the model architecture
model.summary()

Model: "CNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_5 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 56, 56, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 28, 28, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 64)             │     1,605,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,645,830 (6.28 MB)

 Trainable params: 1,645,830 (6.28 MB)

 Non-trainable params: 0 (0.00 B)

In [139]:
#Create a custom callback Class for stoping traing base of the threshold
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, logs=None):
        #will stop the traning if validation accuracy and train accuracy higher than 90% or 99%
        if logs.get('val_accuracy') >= 0.90 or logs.get('accuracy') >= 0.99:
            self.model.stop_training = True

In [147]:
#Create list of callback method
callbacks_list = [
    #model checkpoint for local tracking
    ModelCheckpoint(
        filepath="ckpt/cnn.keras",
        monitor=metrics[0],
        mode="max",
        verbose=1,
        # save_weights_only=True,
        save_best_only=True,
        save_freq="epoch"),

    #model log for every epoch
    WandbMetricsLogger(log_freq="epoch"),

    #model checkpoint for tracking on Wandb
    WandbModelCheckpoint(
        filepath="ckpt/cnn.keras",
        monitor=metrics[0],
        mode="max",
        # save_weights_only=True,
        save_best_only=True,
        save_freq="epoch"),

    #model stopping when reach the threshold
    CustomCallback()

    ##additional callback for experiment
    # ReduceLROnPlateau(
    #     monitor=metrics[0],       # Monitor Validation Loss Metric
    #     factor=0.5,               # Reduce learning rate around 50%
    #     patience=2,               # Reduce Learning Rate if didn't improvement while 5 epoch
    #     verbose=1,                # Showing Log
    #     min_lr=1e-6),               # Learning rate minimum

    # EarlyStopping(
    #     monitor="val_accuracy",
    #     mode="max",
    #     verbose=1,
    #     baseline=0.95,
    #     restore_best_weights=True)

    ]

Back to the Exploratory Data Analysis (EDA), that some problem with imbalanced data. The code below will try to deal with imbalanced data with balancing the class weigth

In [133]:
#Deals with Imbalanced Data

#Store all label
y_train = np.array(ds["label"])

#Compute the class weigth data dan store in array
#Idea of balanced weight is greated the weight of minority class
#Balanced formula: number of label(y_train)/number of the class

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

class_weights = dict(enumerate(class_weights))
print("Class weights:", class_weights)

In [ ]:
#build the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=callbacks_list,
    steps_per_epoch=len(train_ds),
    validation_steps=len(val_ds),
    class_weight=class_weights, #include the class weight
    verbose=1)

In [96]:
#Saving model
path = "./ckpt/cnn.keras" #save modle on local
registered_model_name = "cnn_trashnet_v1" #push model for tacking versioning or experiment to Wandb

run.link_model(path=path, registered_model_name=registered_model_name)
run.finish() #close the running session of Wandb

epoch/accuracy,▁▃▃▄▄▅▆▆▇▇▇▇▇█▇█████████████████████████
epoch/epoch,▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,████████▄▄▄▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,██▇▆▆▃▃▃▂▃▂▂▂▂▁▁▁▁▁▁▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁
epoch/val_accuracy,▁▄▄▄▆▆▆▇████████████████████████████████
epoch/val_loss,█▅▅▅▅▃▄▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,0.61363
epoch/epoch,99
epoch/learning_rate,0.0
epoch/loss,0.9753
epoch/val_accuracy,0.65079
